# Evaluation: Precision & Recall
## Using the evaluation metrics we have learned, we are going to compare how well some different types of classifiers perform on different evaluation metrics
### We are going to use a dataset of written numbers which we can import from sklearn. Run the code below to do so. 
#### N.B. UPDATE!: Instead of running the code below: (1) import: from sklearn.datasets import fetch_openml, and (2) then run: mnist = fetch_openml('mnist_784').

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, plot_roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression

# from sklearn.datasets import fetch_mldata
# mnist = fetch_mldata('MNIST original')

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

X, y = mnist['data'], mnist['target']

### Now take a look at the shapes of the X and y matricies 

In [ ]:
print(X.shape,y.shape)

### Now, let's pick one entry and see what number is written. Use indexing to pick the 36000th digit

In [ ]:
# its not specified if the index is from X or y ? X returns a matrix, y a number...
print(X[36000],y[36000])

### You can use the .reshape(28,28) function and plt.imshow() function with the parameters cmap = matplotlib.cm.binary, interpolation="nearest" to make a plot of the number. Be sure to import matplotlib!

In [ ]:
import matplotlib.pyplot as plt

#has to be X as y cannot be reshapen as its not a matrix.

plt.imshow(X[36000].reshape(28,28), cmap = plt.cm.binary, interpolation="nearest")

# code from Victor above as I could not figure it out...

# seems like the matrix on X has numbers that indicate the intensity of black, which results in a plot 
# that resembles a number. This number coincides with the number on y

# X is a matrix of the image of the number
# y has the number itself

### Use indexing to see if what the plot shows matches with the outcome of the 36000th index

In [ ]:
print(X[36000],y[36000])

### Now lets break into a test train split to run a classification. Instead of using sklearn, use indexing to select the first 60000 entries for the training, and the rest for training.

In [ ]:
# shapes are (70000, 784) (70000,)
X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

# check:
# print(X_train.shape,X_test.shape)

### We are going to make a two-class classifier, so lets restrict to just one number, for example 5s. Do this by defining a new y training and y testing sets for just the number 5

In [ ]:
# no idea what this is asking me to do... what is a 5s?
# and what is a two-class classifier? How do I make it

y_isfive = np.where(y == '5',1,0)
#y_isfive = np.where(y == '5')

print(y_isfive)

y_isfive_train = y_isfive[:60000]
y_isfive_test = y_isfive[60000:]

### Lets train a logistic regression to predict if a number is a 5 or not (remember to use the 'just 5s' y training set!)

In [ ]:
its_a_five = LogisticRegression()

# its_a_five.max_iter= 1000
# all amounts give error, not sure what is a good number of iterations

its_a_five.fit(X_train, y_isfive_train)

y_pred = its_a_five.predict(X)

its_a_five.score(X_test,y_isfive_test)

### Does the classifier predict correctly the 36000th digit we picked before?

In [ ]:
pd.DataFrame({'test':y_isfive, 'predicted':y_pred}).loc[36000]
# prediction is no, because this digit is a 9

### To make some comparisons, we are going to make a very dumb classifier, that never predicts 5s. Build the classifier with the code below, and call it using: never_5_clf = Never5Classifier()

In [ ]:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()

### Now lets fit and predict on the testing set using our never 5 Classifier

In [ ]:
never_5_clf.fit(X_train, y_isfive_train)

y_pred_new = never_5_clf.predict(X_test)


### Let's compare this to the Logistic Regression. Examine the confusion matrix, precision, recall, and f1_scores for each. What is the probability cutoff you are using to decide the classes?

In [ ]:
print(classification_report(y_pred, y_isfive))

### What are the differences you see? Without knowing what each model is, what can these metrics tell you about how well each works?

In [ ]:
# did we see these metrics in class?

### Now let's examine the roc_curve for each. Use the roc_curve method from sklearn.metrics to help plot the curve for each

In [ ]:
# was the roc_curve mentioned in class?


### Now find the roc_auc_score for each. 

In [ ]:
roc_auc_score(y_pred, y_isfive)

### What does this metric tell you? Which classifier works better with this metric in mind?